In [1]:
import weaviate
import json
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from torch import cuda, bfloat16
from langchain.llms import HuggingFacePipeline

from langchain.chains import ChatVectorDBChain,RetrievalQA

In [2]:
import ray

In [3]:
client = weaviate.Client(
    url="http://localhost:8080",   
)

In [ ]:
!pip install ray

In [ ]:
!pip install -U ipywidgets

## Speeding embeddings using Ray and Weaviate

In [3]:
import ray

In [4]:
ray.init()

/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-24_10-09-57_317390_1715346/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-24_10-09-57_317390_1715346/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-24_10-09-57_317390_1715346/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10

Python version:,3.8.10
Ray version:,2.7.1


In [5]:
ray.init(num_cpus=16, num_gpus=1)

/home/ubuntu/LLM-as-a-Service/weaviate_env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-30_15-48-18_397248_1914599/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/weaviate_env/lib/python3.8/site-packages/ray/_private/node.py:1403: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-30_15-48-18_397248_1914599/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/ubuntu/LLM-as-a-Service/weaviate_env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2023-10-30_15-48-18_397248_1914599/logs/monitor.out' mode='a' encoding='utf-8'>
  self.start_monitor()
/home/ubuntu/LLM-as-a-Service/weaviate_env/lib/python3.8/site-packages/ray/_private/node.py:1408: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/ses

Python version:,3.8.10
Ray version:,2.7.1


In [4]:
ray.shutdown()

In [27]:
!pip install pyarrow

     |████████████████████████████████| 40.1 MB 12.8 MB/s eta 0:00:01


In [6]:
# imports
from ray.data.datasource import FileExtensionFilter
import io
from pypdf import PdfReader
import pypdf
import binascii

from langchain.text_splitter import RecursiveCharacterTextSplitter

import transformers

# Ray and paralized embeddings

## Embedding class

* Init the embedding model.
* Give the class the functions to create the embeddings. 
* Give maybe parameters to assign only fractions of GPU. 


In [15]:
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [7]:
@ray.remote
class EmbeddingActor:
    def __init__(self):
        self.lst_embeddings = []
        self.embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-xl",)
        
    def get_embeddings(self,texts):
        if not isinstance(texts, list):
            texts = [texts]
        # Ensure each text is a string
        texts = [str(text) for text in texts]
        
        lst_embeddings = self.embeddings.embed_documents(texts)
        return lst_embeddings
    
    def get(self):
        return self.lst_embeddings


        #self.model = transformers.pipeline("feature-extraction", model=self.model_name, device=0)
                    

## Splitting the doc

In [8]:
loader = PyPDFLoader("../pdf_querying_summarization/temp_file/doc.pdf")

documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

docs = text_splitter.split_documents(documents)

## Approach 1 (Ray and embedding model)

### Single actor:

In [ ]:
act = EmbeddingActor.remote()

(EmbeddingActor pid=1905266) load INSTRUCTOR_Transformer


(EmbeddingActor pid=1905266) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(EmbeddingActor pid=1905266)   warn("The installed version of bitsandbytes was compiled without GPU support. "


(EmbeddingActor pid=1905266) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [10]:
!time
obj_ref = act.get_embeddings.remote(docs)
ray.get(obj_ref)


real	0m0.000s
user	0m0.000s
sys	0m0.000s


[[0.053445033729076385,
  -0.023476049304008484,
  0.036313023418188095,
  0.009126272983849049,
  -0.03635646030306816,
  -0.06295235455036163,
  -0.07791279256343842,
  0.03922927752137184,
  -0.010391725227236748,
  -0.021906543523073196,
  0.0028830531518906355,
  0.056121498346328735,
  0.008307699114084244,
  -0.16414985060691833,
  -0.023184409365057945,
  0.029710976406931877,
  0.01872403733432293,
  -0.04861781373620033,
  -0.036475516855716705,
  0.028931858018040657,
  -0.0060825832188129425,
  -0.006084576714783907,
  -0.04779592901468277,
  0.005550459958612919,
  -0.015297983773052692,
  -0.06648053973913193,
  0.002558498177677393,
  0.030010532587766647,
  -0.03353802114725113,
  -0.021990429610013962,
  0.04802055284380913,
  -0.0006715736235491931,
  -0.011839083395898342,
  -0.05323244631290436,
  0.0366913378238678,
  -0.000987737555988133,
  -0.020954228937625885,
  -0.04361029714345932,
  0.03991300240159035,
  -0.015399004332721233,
  -0.0331735759973526,
  0.01

Took 1 min 40 sec to embed the document on the GRI standards using a single actor and a single CPU.

### Multiple actors

In [8]:
actors = [EmbeddingActor.remote() for _ in range(3)]

results = [actor.get_embeddings.remote(docs) for actor in actors]

ray.get(results)

!time

(EmbeddingActor pid=1908160) load INSTRUCTOR_Transformer
(EmbeddingActor pid=1908159) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


(EmbeddingActor pid=1908158) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
(EmbeddingActor pid=1908158)   warn("The installed version of bitsandbytes was compiled without GPU support. "
(EmbeddingActor pid=1908158) Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
(EmbeddingActor pid=1908160) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizer

(EmbeddingActor pid=1908159) max_seq_length  512
(EmbeddingActor pid=1908158) load INSTRUCTOR_Transformer [repeated 2x across cluster]
(EmbeddingActor pid=1908160) /home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32 [repeated 2x across cluster]

real	0m0.000s
user	0m0.000s
sys	0m0.000s


In [51]:
ray.get(results)

[[[0.053445033729076385,
   -0.023476049304008484,
   0.036313023418188095,
   0.009126272983849049,
   -0.03635646030306816,
   -0.06295235455036163,
   -0.07791279256343842,
   0.03922927752137184,
   -0.010391725227236748,
   -0.021906543523073196,
   0.0028830531518906355,
   0.056121498346328735,
   0.008307699114084244,
   -0.16414985060691833,
   -0.023184409365057945,
   0.029710976406931877,
   0.01872403733432293,
   -0.04861781373620033,
   -0.036475516855716705,
   0.028931858018040657,
   -0.0060825832188129425,
   -0.006084576714783907,
   -0.04779592901468277,
   0.005550459958612919,
   -0.015297983773052692,
   -0.06648053973913193,
   0.002558498177677393,
   0.030010532587766647,
   -0.03353802114725113,
   -0.021990429610013962,
   0.04802055284380913,
   -0.0006715736235491931,
   -0.011839083395898342,
   -0.05323244631290436,
   0.0366913378238678,
   -0.000987737555988133,
   -0.020954228937625885,
   -0.04361029714345932,
   0.03991300240159035,
   -0.015399004

Took 1.13 minutes to embed the document on the GRI standards 3 times on 3 different CPUs. 
Conclusion: The paralellization works.

### Multiple agents on the same document

In [9]:
### divide the documents for each actors

def divide_workload(num_actors, documents):
    docs_per_actor = len(documents) // num_actors

    doc_parts = [docs[i * docs_per_actor: (i + 1) * docs_per_actor] for i in range(num_actors)]

    if len(docs) % num_actors:
        doc_parts[-1].extend(docs[num_actors * docs_per_actor:])

    return doc_parts

In [10]:
num_actors = 6
doc_parts = divide_workload(num_actors, docs)

In [ ]:
!timeit
actors = [EmbeddingActor.remote() for _ in range(num_actors)]

results = [actor.get_embeddings.remote(doc_part) for actor, doc_part in zip(actors, doc_parts)]

In [63]:
ray.get(results)

(EmbeddingActor pid=1762429) load INSTRUCTOR_Transformer
(EmbeddingActor pid=1762432) max_seq_length  512
(EmbeddingActor pid=1762433) load INSTRUCTOR_Transformer [repeated 5x across cluster]


[[[0.05344502255320549,
   -0.02347605675458908,
   0.0363130159676075,
   0.009126272983849049,
   -0.03635644167661667,
   -0.06295236200094223,
   -0.07791279256343842,
   0.03922928124666214,
   -0.010391724295914173,
   -0.021906528621912003,
   0.002883058274164796,
   0.05612149462103844,
   0.00830770656466484,
   -0.16414982080459595,
   -0.023184416815638542,
   0.02971096895635128,
   0.018724026158452034,
   -0.04861781373620033,
   -0.036475520581007004,
   0.028931858018040657,
   -0.00608258880674839,
   -0.006084572523832321,
   -0.04779593646526337,
   0.0055504534393548965,
   -0.015297993086278439,
   -0.06648054718971252,
   0.0025584970135241747,
   0.030010513961315155,
   -0.03353802487254143,
   -0.021990418434143066,
   0.048020556569099426,
   -0.0006715698982588947,
   -0.011839083395898342,
   -0.05323244631290436,
   0.036691319197416306,
   -0.0009877359261736274,
   -0.02095421962440014,
   -0.04361029341816902,
   0.03991299122571945,
   -0.0153990043327

It takes around 53 seconds to embed the GRI document when using 3 CPUs, 3 actors and dividing the workload on each actors equally. And 32 seconds when using 6 actors with equal workload.

>7655 root       992M  159M 40256 S  0.6  0.1  1h59:03 /bin/weaviate -51620   0 31.9G  466M 51424 S  0.6  0.2  2:25.00 /home/ubuntu/.v 634576 root       20   0  992M  159M 40256 S  0.6  0.1  2h06:39 /bin/weaviate -17;27H22.1G  528M  202M 3  1:41.96 /home/ubuntu/LL 156293 root      2930M 36408  8660 1h00:15 /usr/bin/docker81714.6G 45260 16148 S  1.9  0.0  5:49.48LL 684945 root       992M  159M 40256 S  1.3  0.1  2h01:14 /bin/weaviate -6522.1G  528M  202M S  1.3  0.3  0:00.89 /home/ubuntu/LL34581h57:17H760 root       RT   0  273M 17948  8208 S  0.0  0.0  0:06.38 /sbin/multipath761 root       RT   0  273M 17948  8208 S  0.0  0.0  4:33.42 /sbin/multipath762 root       RT   0  273M 17948  8208 S  0.0  0.0  0:00.00 /sbin/multipath763 root       RT   0  273M 17948  8208 S  0.0  0.0  0:00.00 /sbin/multipathF1Help  F2Setup F3SearchF4FilterF5Tree  F6SortByF7Nice -F8Nice +F9Kill  F10Quit|||    3.9||     1.3||     2.6||     1.3|1.9|6|7|7    11.2%||     2|7|7|      1.3%|7||     1.9||     2||     4

In [23]:
actors = [EmbeddingActor.remote() for _ in range(3)]

results = [actor.get_embeddings.remote(docs) for actor in actors]

## Trying parallelization on GPUs

In [70]:
@ray.remote(num_gpus=0.2)
class EmbeddingActor:
    def __init__(self):
        self.lst_embeddings = []
        self.embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-xl",)
        
    def get_embeddings(self,texts):
        if not isinstance(texts, list):
            texts = [texts]
        # Ensure each text is a string
        texts = [str(text) for text in texts]
        
        lst_embeddings = self.embeddings.embed_documents(texts)
        return lst_embeddings
    
    def get(self):
        return self.lst_embeddings


In [71]:
num_actors = 3
doc_parts = divide_workload(num_actors, docs)

In [72]:
actors = [EmbeddingActor.remote() for _ in range(num_actors)]

results = [actor.get_embeddings.remote(doc_part) for actor, doc_part in zip(actors, doc_parts)]

(EmbeddingActor pid=1769240) load INSTRUCTOR_Transformer


In [73]:
ray.get(results)

(EmbeddingActor pid=1769240) max_seq_length  512
(EmbeddingActor pid=1769239) load INSTRUCTOR_Transformer [repeated 2x across cluster]


[[[0.05344506353139877,
   -0.023476067930459976,
   0.03631303459405899,
   0.009126313962042332,
   -0.03635643422603607,
   -0.06295238435268402,
   -0.07791274785995483,
   0.03922925144433975,
   -0.01039165910333395,
   -0.021906554698944092,
   0.0028830384835600853,
   0.05612153932452202,
   0.008307748474180698,
   -0.16414983570575714,
   -0.023184416815638542,
   0.029710952192544937,
   0.01872403547167778,
   -0.048617806285619736,
   -0.036475490778684616,
   0.02893185429275036,
   -0.006082591135054827,
   -0.006084568332880735,
   -0.047795917838811874,
   0.0055504231713712215,
   -0.015297985635697842,
   -0.06648051738739014,
   0.0025584844406694174,
   0.030010512098670006,
   -0.033538032323122025,
   -0.02199041284620762,
   0.048020534217357635,
   -0.0006715664640069008,
   -0.011839048936963081,
   -0.05323244258761406,
   0.036691270768642426,
   -0.0009877299889922142,
   -0.020954227074980736,
   -0.04361020773649216,
   0.03991297632455826,
   -0.0153990

When the actors run on a slice of GPU (0.2) the performance increases drastically. It now takes 12 to 13 seconds to do the embeddings considering that we are using the same GPU

## Approach 2 (Ray and Weaviate auto embedder)

### Creating the weaviate embedding class

In [64]:
from weaviate.util import generate_uuid5

@ray.remote(num_gpus=0.2)
class WeaviateEmbedder:
    def __init__(self):
        self.weaviate_client = weaviate.Client(
            url="http://localhost:8080",   
        )

    def adding_weaviate_document(self, text_lst, collection_name):

        self.weaviate_client.batch.configure(batch_size=50)

        with self.weaviate_client.batch as batch:
            for text in text_lst:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
                        uuid=generate_uuid5(text),
        )

    def get(self):
        return self.lst_embeddings

### Splitting the document and workload

In [65]:
def weaviate_serialize_document(doc, title):
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

In [66]:
def split_document(docs, doc_name):        

        loader = PyPDFLoader(docs)

        documents = loader.load()

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

        text_docs = text_splitter.split_documents(documents)
        serialized_docs = [
            weaviate_serialize_document(doc,doc_name) 
            for doc in text_docs
            ]
        return serialized_docs	


In [67]:
### divide the documents for each actors

def divide_workload(num_actors, documents):
    docs_per_actor = len(documents) // num_actors

    doc_parts = [documents[i * docs_per_actor: (i + 1) * docs_per_actor] for i in range(num_actors)]

    if len(documents) % num_actors:
        doc_parts[-1].extend(documents[num_actors * docs_per_actor:])

    return doc_parts

In [68]:
serialized_docs = split_document("../pdf_querying_summarization/temp_file/doc.pdf", "doc")

In [69]:
serialized_docs

[{'page_content': 'A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org',
  'document_title': 'doc'},
 {'page_content': '2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThree se

In [70]:
num_actors = 3

In [71]:
doc_parts = divide_workload(num_actors, serialized_docs)

In [72]:
doc_parts

[[{'page_content': 'A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org',
   'document_title': 'doc'},
  {'page_content': '2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThree

In [73]:
actors = [WeaviateEmbedder.remote() for _ in range(num_actors)]

results = [actor.adding_weaviate_document.remote(doc_part, "WokToWalk") for actor, doc_part in zip(actors, doc_parts)]

In [38]:
ray.get(results)

[None, None, None]

### Checking Weaviate Database

In [5]:
weaviate_client = weaviate.Client(
            url="http://localhost:8080",   
        )

In [63]:
schema = {'classes': [ 
            {
                    'class': 'WokToWalk',
                    'description': 'normal description',
                    'vectorizer': 'text2vec-transformers',
                    'moduleConfig': {
                        'text2vec-transformers': {
                            'vectorizerClassName': False,
                            }
                    },
                    'properties': [{
                        'dataType': ['text'],
                        'description': 'the text from the documents parsed',
                        'moduleConfig': {
                            'text2vec-transformers': {
                                'skip': False,
                                'vectorizePropertyName': False,
                                }
                        },
                        'name': 'page_content',
                    },
                    {
                        'name': 'document_title',
                        'dataType': ['text'],
                    }],      
                    },
        ]}


weaviate_client.schema.create(schema)

In [74]:
weaviate_client.data_object.get(class_name="WokToWalk")

{'deprecations': [],
 'objects': [{'class': 'WokToWalk',
   'creationTimeUnix': 1698328229793,
   'id': '03739613-a4ea-54f0-88d9-04f18d9e46ed',
   'lastUpdateTimeUnix': 1698328229793,
   'properties': {'document_title': 'doc',
    'page_content': "3\nA Short Introduction to the GRI Standards\n...\nApply all three Universal \nStandards to your reportingUse the Sector Standards that \napply to your sectorsSelect Topic Standards to report \nspeciﬁc information on your \nmaterial topicsSector Standards Universal Standards\n...Topic Standards\nGRI 201\nGRI 415\nGRI 304\nGRI 403\nGRI 303\nGRI 205\nGRI 305\nGRI 202\nGRI 13\nGRI 16\nGRI 12\nGRI 15\nGRI 18\nGRI 11\nGRI 14\nGRI 17GRI Standards\nRequirements and \nprinciples for using the \nGRI Standards\nDisclosures about the \nreporting organization\nDisclosures and \nguidance about the \norganization's material \ntopics \nGRI 1\nGRI 2\nGRI 3Figure 1. GRI Standards: Universal, Sector and Topic Standards• GRI 2: General Disclosures 2021 ( GRI 2 

### Ray random tests

In [40]:
import ray
import random
import time
import math
from fractions import Fraction

In [48]:
@ray.remote
def pi4_sample(sample_count):
    """pi4_sample runs sample_count experiments, and returns the 
    fraction of time it was inside the circle. 
    """
    in_count = 0
    for i in range(sample_count):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            in_count += 1
    return Fraction(in_count, sample_count)

In [49]:
SAMPLE_COUNT = 1000 * 1000
start = time.time() 
future = pi4_sample.remote(sample_count = SAMPLE_COUNT)
pi4 = ray.get(future)
end = time.time()
dur = end - start
print(f'Running {SAMPLE_COUNT} tests took {dur} seconds')

Running 1000000 tests took 0.301433801651001 seconds


In [50]:
pi = pi4 * 4

In [51]:
float(pi)

3.144568

In [ ]:
FULL_SAMPLE_COUNT = 100 * 1000 * 1000 * 1000 # 100 billion samples! 
BATCHES = int(FULL_SAMPLE_COUNT / SAMPLE_COUNT)
print(f'Doing {BATCHES} batches')
results = []
for _ in range(BATCHES):
    results.append(pi4_sample.remote(sample_count = SAMPLE_COUNT))
output = ray.get(results)

In [56]:
import os

In [57]:
os.cpu_count()

24

### Random idea to use and process ray dataset

In [41]:
ds = ray.data.from_items(docs)

In [49]:
ds = ds.map_batches(
    Embed,
    # Large batch size to maximize GPU utilization.
    # Too large a batch size may result in GPU running out of memory.
    # If the chunk size is increased, then decrease batch size.
    # If the chunk size is decreased, then increase batch size.
    batch_size=100,  # Large batch size to maximize GPU utilization.
    compute=ray.data.ActorPoolStrategy(min_size=1, max_size=1),  # I have 20 GPUs in my cluster
    num_gpus=0.25,  # 1 GPU for each actor.
)

# Weaviate functions

In [7]:
def weaviate_serialize_document(doc, title):
        
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

In [9]:
serialized_docs = [
            weaviate_serialize_document(doc,"GRI_STD") 
            for doc in docs
            ]

In [10]:
serialized_docs

[{'page_content': 'A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org',
  'document_title': 'GRI_STD'},
 {'page_content': '2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThre

## Weaviate Schema creation

In [29]:
schema = {'classes': [ 
            {
                    'class': 'test_ray',
                    'description': 'normal description',
                    'vectorizer': 'text2vec-transformers',
                    'moduleConfig': {
                        'text2vec-transformers': {
                            'vectorizerClassName': False,
                            }
                    },
                    'properties': [{
                        'dataType': ['text'],
                        'description': 'the text from the documents parsed',
                        'moduleConfig': {
                            'text2vec-transformers': {
                                'skip': False,
                                'vectorizePropertyName': False,
                                }
                        },
                        'name': 'page_content',
                    },
                    {
                        'name': 'document_title',
                        'dataType': ['text'],
                    }],      
                    },
        ]}


client.schema.create(schema)

UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "Test_ray" already exists'}]}.

## Batching the data

In [16]:
def process_batch_embeddings(serialized, collection_name):    
    client.batch.configure(batch_size=50)

    with client.batch as batch:
                for text in serialized:
                        batch.add_data_object(
                            text,
                            class_name=collection_name, 
            )

In [26]:
client.data_object.get()

{'deprecations': None,
 'objects': [{'class': 'Test_with_vectorizer',
   'creationTimeUnix': 1696427776033,
   'id': '07070486-7830-460a-bf85-722f1fc44ed4',
   'lastUpdateTimeUnix': 1696427776033,
   'properties': {'page_content': "3\nA Short Introduction to the GRI Standards\n...\nApply all three Universal \nStandards to your reportingUse the Sector Standards that \napply to your sectorsSelect Topic Standards to report \nspeciﬁc information on your \nmaterial topicsSector Standards Universal Standards\n...Topic Standards\nGRI 201\nGRI 415\nGRI 304\nGRI 403\nGRI 303\nGRI 205\nGRI 305\nGRI 202\nGRI 13\nGRI 16\nGRI 12\nGRI 15\nGRI 18\nGRI 11\nGRI 14\nGRI 17GRI Standards\nRequirements and \nprinciples for using the \nGRI Standards\nDisclosures about the \nreporting organization\nDisclosures and \nguidance about the \norganization's material \ntopics \nGRI 1\nGRI 2\nGRI 3Figure 1. GRI Standards: Universal, Sector and Topic Standards• GRI 2: General Disclosures 2021 ( GRI 2 ) contains \ndis

In [ ]:
def weaviate_serialize_document(doc, title):
        
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

def adding_weaviate_document(docs, collection_name, doc_name):
        loader = PyPDFLoader(docs)

        documents = loader.load()

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

        text_docs = text_splitter.split_documents(documents)
        serialized_docs = [
            weaviate_serialize_document(doc,doc_name) 
            for doc in text_docs
            ]
        client.batch.configure(batch_size=50)

        with client.batch as batch:
            for text in serialized_docs:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
        )

In [ ]:
def weaviate_serialize_document(doc, title):
        
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

def adding_weaviate_document(docs, collection_name, doc_name):
        loader = PyPDFLoader(docs)

        documents = loader.load()

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

        text_docs = text_splitter.split_documents(documents)
        serialized_docs = [
            weaviate_serialize_document(doc,doc_name) 
            for doc in text_docs
            ]
        client.batch.configure(batch_size=50)

        with client.batch as batch:
            for text in serialized_docs:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
        )

In [5]:
def weaviate_serialize_document(doc, title):
        
        return {
            "page_content": doc.page_content,
            "document_title": title,
        }

In [8]:
#weaviate_serialize_document(docs, "test")
serialized_docs = [
            weaviate_serialize_document(doc,"test") 
            for doc in docs
            ]

In [11]:
def adding_weaviate_document(docs, collection_name, doc_name):
        loader = PyPDFLoader(docs)

        documents = loader.load()

        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

        text_docs = text_splitter.split_documents(documents)
        serialized_docs = [
            weaviate_serialize_document(doc,doc_name) 
            for doc in text_docs
            ]
        client.batch.configure(batch_size=50)

        with client.batch as batch:
            for text in serialized_docs:
                    batch.add_data_object(
                        text,
                        class_name=collection_name, 
        )

In [19]:
adding_weaviate_document("../pdf_querying_summarization/temp_file/doc.pdf", "test_delete_class", "GRI_standards")

In [16]:
def delete_weaviate_document(name, cls_name):
        document_name = str(name)
        client.batch.delete_objects(
            class_name=cls_name,
            where={
                "path": ["document_title"],
                "operator": "Like",
                "valueText": document_name,
            }
        )

In [17]:
delete_weaviate_document("The_GRI_standards","New_class_to_test_delete")

In [20]:
class_properties = ["document_title"]
query = client.query.get("test_delete_class",class_properties)

In [24]:
def query_weaviate_document_names(cls, class_properties):
        query = client.query.get(cls, class_properties)
        query = query.do()

        document_title_set = set()
        documents = query.get('data', {}).get('Get', {}).get('Test_delete_class', [])

        for document in documents:
            document_title = document.get('document_title')
            if document_title is not None:
                document_title_set.add(document_title)
        return list(document_title_set)

In [25]:
query_weaviate_document_names("test_delete_class", ["document_title"])

['GRI_standards']

In [23]:
query.do()

{'data': {'Get': {'Test_delete_class': [{'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'},
    {'document_title': 'GRI_standards'}]}}}

In [7]:
vectorstore = Weaviate(weaviate_client, 'WokToWalk', 'page_content', attributes=['page_content'])

In [12]:
text_metadata = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_metadata))
vectorstore.add_texts(texts, meta)

['1ca09ed8-cd2f-452f-9da0-d5efa5c3e145',
 '5bc65713-3bbe-47d1-8ed5-1172a2f56997',
 '0248871a-7c6c-41fb-91c2-d5bd32b36ecd',
 '915fbeb1-d4c1-4f4d-a570-fc178fc4ef4f',
 'ef5612ad-034c-409e-8393-51842d1616a3',
 '0572cd2f-e49c-4b78-b8a4-0529bc3bda5f']

In [30]:
text_doc = [(doc.page_content) for doc in docs]

In [33]:
def serialize_document(doc):
    return {
        "page_content": doc.page_content,
    }

In [35]:
serialized_docs = [
    serialize_document(doc) 
    for doc in docs
]

In [36]:
serialized_docs

[{'page_content': 'A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org'},
 {'page_content': '2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThree series of Standards support t

In [31]:
text_doc

['A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org',
 '2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThree series of Standards support the reporting process: \nthe GRI Top

In [77]:
query = "What are the GRI standards?"
query_results = vectorstore.similarity_search(query, top_k=20)

In [78]:
query_results

[Document(page_content='A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org', metadata={}),
 Document(page_content='2\nA Short Introduction to the GRI Standards\nIntroduction\nThe GRI Standards are a modular system \nof interconnected standards. They allow \norganizations to publicly report the \nimpacts of their activities in a structured \nway that is transparent to stakeholders \nand other interested parties.\nThis Short Introduction will:\n•  give new users of the GRI Standards an overview \nof how the Standards are set up, and equip them to \nstart working with the various elements involved in the \nreporting process;\n•  be of assistance to experienced users in gaining an \nunderstanding of changes in the system and the role of \nthe GRI Sector Standards; and\n•  aid stakeholders and other information users (such \nas analysts and policymakers) to understand how the \nreporting process works and what to look for in a report.\nThree series

In [80]:
!pip install bitsandbytes

  Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)


In [82]:
!pip install accelerate

     |████████████████████████████████| 260 kB 13.3 MB/s eta 0:00:01


In [6]:
model_id = 'meta-llama/Llama-2-70b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

        # set quantization configuration to load large model with less GPU memory
        # this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=bfloat16
        )

        # begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
            model_id
        )

model = transformers.AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            config=model_config,
            quantization_config=bnb_config,
            device_map='auto'
        )
model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
generate_text = transformers.pipeline(
            model=model, tokenizer=tokenizer,
            return_full_text=True,  # langchain expects the full text
            task='text-generation',
            # we pass model parameters here too
            #stopping_criteria=stopping_criteria,  # without this model rambles during chat
            temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
            max_new_tokens=512,  # mex number of tokens to generate in the output
            repetition_penalty=1.1  # without this output begins repeating
        )
llm = HuggingFacePipeline(pipeline=generate_text)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [17]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(
    llm,chain_type='stuff'
)

In [10]:
QA_document = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [11]:
QA_document.run("what are the GRI standards?")

' The GRI standards are a modular system of interconnected standards that allow organizations to publicly report the impacts of their activities in a structured way that is transparent to stakeholders and other interested parties.'

In [21]:
client.data_object.get(class_name="Chatbot")

{'deprecations': [],
 'objects': [{'class': 'Chatbot',
   'creationTimeUnix': 1696511468359,
   'id': '0248871a-7c6c-41fb-91c2-d5bd32b36ecd',
   'lastUpdateTimeUnix': 1696511468359,
   'properties': {'page': 2,
    'page_content': "3\nA Short Introduction to the GRI Standards\n...\nApply all three Universal \nStandards to your reportingUse the Sector Standards that \napply to your sectorsSelect Topic Standards to report \nspeciﬁc information on your \nmaterial topicsSector Standards Universal Standards\n...Topic Standards\nGRI 201\nGRI 415\nGRI 304\nGRI 403\nGRI 303\nGRI 205\nGRI 305\nGRI 202\nGRI 13\nGRI 16\nGRI 12\nGRI 15\nGRI 18\nGRI 11\nGRI 14\nGRI 17GRI Standards\nRequirements and \nprinciples for using the \nGRI Standards\nDisclosures about the \nreporting organization\nDisclosures and \nguidance about the \norganization's material \ntopics \nGRI 1\nGRI 2\nGRI 3Figure 1. GRI Standards: Universal, Sector and Topic Standards• GRI 2: General Disclosures 2021 ( GRI 2 ) contains \ndis

In [18]:
chain.run(input_documents=docs, question=query)

" The GRI standards are a modular system of interconnected standards that allow organizations to publicly report the impacts of their activities in a structured way that is transparent to stakeholders and other interested parties. They are designed to help organizations report on their sustainability performance in a consistent and comparable manner, and to provide stakeholders with accurate and reliable information about an organization's impacts on the economy, environment, and people. The standards are divided into three series: Universal Standards, Sector Standards, and Topic Standards."

In [15]:
db = Weaviate.from_documents(docs, embeddings, weaviate_url="http://localhost:8080", by_text=False)

In [12]:
query = "When were the GRI standards written?"
docs = db.similarity_search(query)

In [13]:
docs

[Document(page_content="3\nA Short Introduction to the GRI Standards\n...\nApply all three Universal \nStandards to your reportingUse the Sector Standards that \napply to your sectorsSelect Topic Standards to report \nspeciﬁc information on your \nmaterial topicsSector Standards Universal Standards\n...Topic Standards\nGRI 201\nGRI 415\nGRI 304\nGRI 403\nGRI 303\nGRI 205\nGRI 305\nGRI 202\nGRI 13\nGRI 16\nGRI 12\nGRI 15\nGRI 18\nGRI 11\nGRI 14\nGRI 17GRI Standards\nRequirements and \nprinciples for using the \nGRI Standards\nDisclosures about the \nreporting organization\nDisclosures and \nguidance about the \norganization's material \ntopics \nGRI 1\nGRI 2\nGRI 3Figure 1. GRI Standards: Universal, Sector and Topic Standards• GRI 2: General Disclosures 2021 ( GRI 2 ) contains \ndisclosures relating to details about an organization’s \nstructure and reporting practices; activities and \nworkers; governance; strategy; policies; practices; and \nstakeholder engagement. These give insight 

In [8]:
docs = db.similarity_search_with_score(query, by_text=False)
docs[0]

(Document(page_content='A Short Introduction to  \nthe GRI Standards\nwww.globalreporting.org www.globalreporting.org', metadata={'_additional': {'vector': [0.053567585, -0.0251535, 0.019101305, 0.0066484925, -0.062171742, -0.04195238, -0.088961646, 0.029081617, -0.009508389, -0.005521557, 0.036933485, 0.038459525, -0.013330513, -0.10212877, -0.023793692, 0.03594412, 0.007122637, -0.039632343, -0.010635164, -0.01424749, 0.0057760137, -0.04446114, -0.041216586, 0.026710706, -0.014804762, -0.054996327, 0.023391552, 0.013136665, -0.036958776, -0.03909738, 0.052267905, -0.011480347, 0.0067614955, -0.046085354, 0.04469198, -0.009438213, -0.023576789, -0.022953432, 0.0030165846, -0.0012048823, -0.06619726, 0.025584906, 0.017831862, -0.0054667275, 0.0027591179, -0.0104497885, -0.0023296648, 0.019857239, -0.019289603, -0.03826433, -0.013847204, -0.053075723, 0.021076266, -0.03891018, 0.017620891, 0.024064133, -0.026787126, -0.0139441285, -0.036185306, 0.05172352, 0.024646377, 0.021141836, 0.00

In [16]:
db._client.schema.get()

{'classes': [{'class': 'Vec',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-transformers': {'poolingStrategy': 'masked_mean',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Tue Sep 12 14:44:58 2023",
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-transformers': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'answer',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "This property was generated by Weaviate's auto-schema feature on Tue Sep 12 14:44:58 2023",
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-transformers': {'skip': False,
       've

# Vectorstore and LLM test

Creating the vectorstore to query information from the DB using the LLM

In [11]:
vectorstore = Weaviate(client, "LangChain_138df2f1432f4c5c825cff1cf8d79ee9", "text")

Init LLM

In [15]:
model_id = 'meta-llama/Llama-2-70b-chat-hf'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

        # set quantization configuration to load large model with less GPU memory
        # this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type='nf4',
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=bfloat16
        )

        # begin initializing HF items, need auth token for these
model_config = transformers.AutoConfig.from_pretrained(
            model_id
        )

model = transformers.AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            config=model_config,
            quantization_config=bnb_config,
            device_map='auto'
        )
model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
generate_text = transformers.pipeline(
            model=model, tokenizer=tokenizer,
            return_full_text=True,  # langchain expects the full text
            task='text-generation',
            # we pass model parameters here too
            #stopping_criteria=stopping_criteria,  # without this model rambles during chat
            temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
            max_new_tokens=512,  # mex number of tokens to generate in the output
            repetition_penalty=1.1  # without this output begins repeating
        )
llm = HuggingFacePipeline(pipeline=generate_text)

Loading checkpoint shards: 100%|██████████| 15/15 [00:16<00:00,  1.11s/it]


Creating the chains: 

In [12]:
qa = ChatVectorDBChain.from_llm(llm, vectorstore)
chat_history = []

/home/ubuntu/LLM-as-a-Service/llama-env/lib/python3.8/site-packages/langchain/chains/conversational_retrieval/base.py:295: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warnings.warn(


In [15]:
QA_document = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [16]:
QA_document.run("what are the gri standards?")

ValueError: Error during query: [{'locations': [{'column': 6, 'line': 1}], 'message': "explorer: get class: vector search: object vector search at index langchain_138df2f1432f4c5c825cff1cf8d79ee9: shard langchain_138df2f1432f4c5c825cff1cf8d79ee9_jqTX3xTHTaox: vector search: knn search: distance between entrypoint and query node: vector lengths don't match: 768 vs 384", 'path': ['Get', 'LangChain_138df2f1432f4c5c825cff1cf8d79ee9']}]

In [13]:
query = 'What are the GRI standards?'

result = qa({'question':query,'chat_history': chat_history})

ValueError: Error during query: [{'locations': [{'column': 6, 'line': 1}], 'message': "explorer: get class: vector search: object vector search at index langchain_138df2f1432f4c5c825cff1cf8d79ee9: shard langchain_138df2f1432f4c5c825cff1cf8d79ee9_jqTX3xTHTaox: vector search: knn search: distance between entrypoint and query node: vector lengths don't match: 768 vs 384", 'path': ['Get', 'LangChain_138df2f1432f4c5c825cff1cf8d79ee9']}]

# Weaviate database management functions

* Create Schema
* Create Class
* Add data with custom metadata, name, description etc...
* Query data
* Remove data
* Update data

In [ ]:
#Schema and class creation: 

class_name = ""
vectorizer = ""


## Random schemas

In [ ]:
schema = {'classes': [ 
            {
                    'class': 'test_ray',
                    'description': 'normal description',
                    'vectorizer': 'text2vec-transformers',
                    'moduleConfig': {
                        'text2vec-transformers': {
                            'vectorizerClassName': False,
                            }
                    },
                    'properties': [{
                        'dataType': ['text'],
                        'description': 'the text from the documents parsed',
                        'moduleConfig': {
                            'text2vec-transformers': {
                                'skip': False,
                                'vectorizePropertyName': False,
                                }
                        },
                        'name': 'page_content',
                    },
                    {
                        'name': 'document_title',
                        'dataType': ['text'],
                    }],      
                    },
        ]}


client.schema.create(schema)